In [2]:
import pandas as pd
import numpy as np
from PIL import Image,ImageFilter
import cv2 as cv
import matplotlib.pyplot as plt
import heapq
from tqdm import tqdm
import sys

In [82]:
# 首先看图片大小是否一样,不是很好存
with open('data/dataset/manmade_training.txt','r',encoding='utf-8') as f:
    im_name = [line.strip('\n') for line in f.readlines()]

size = np.zeros((len(im_name),2))  
cnt = 0
for path in im_name:
    path = 'data/dataset/' + path
    im = np.array(Image.open(path))
    size[cnt] = np.shape(im[:,:,0])
    cnt += 1

size



array([[1024.,  768.],
       [1290., 2032.],
       [1244.,  877.],
       [ 851., 1280.],
       [2200., 1650.],
       [ 240.,  320.],
       [ 400.,  600.],
       [ 224.,  400.],
       [ 600.,  800.],
       [1024., 1536.],
       [ 330.,  250.],
       [2304., 3072.],
       [ 480.,  640.],
       [ 545.,  741.],
       [ 275.,  204.],
       [2448., 3264.],
       [1704., 2272.],
       [ 316.,  200.],
       [ 960., 1280.],
       [ 600.,  800.],
       [ 256.,  256.],
       [ 768., 1024.],
       [ 384.,  256.],
       [ 202.,  310.],
       [2112., 2816.],
       [1577., 2221.],
       [ 300.,  224.],
       [ 256.,  256.],
       [1536., 2048.],
       [ 256.,  256.],
       [ 480.,  640.],
       [ 402.,  600.],
       [ 240.,  320.],
       [2304., 3456.],
       [ 800.,  481.],
       [ 256.,  256.],
       [2592., 1944.],
       [1368., 1712.],
       [ 549.,  732.],
       [1320.,  960.],
       [ 240.,  320.],
       [ 240.,  320.],
       [2448., 3264.],
       [ 96

In [125]:
# This function will get the pixels of each partition

def get_pixels(num_pixels_x, num_tiles_x):
    tile_pixels_x = np.zeros(3)
    if num_pixels_x % num_tiles_x == 0:
        tile_pixels_x = num_pixels_x/num_tiles_x
    else:
        tile_pixels_x[2] = int(num_pixels_x/num_tiles_x)
        remain = num_pixels_x % num_tiles_x
        if remain % 2 == 0:
            tile_pixels_x[0] = tile_pixels_x[1] + remain/2
            tile_pixels_x[2] = tile_pixels_x[1] + remain/2
        else:
            tile_pixels_x[0] = tile_pixels_x[1] + int(remain/2)
            tile_pixels_x[2] = tile_pixels_x[1] + int(remain/2) + 1

    x1 = np.zeros(num_tiles_x + 1)
    x1[1] = tile_pixels_x[0]
    x1[2:-1] = tile_pixels_x[1]
    x1[-1] = tile_pixels_x[2]

    return x1


# This method will randomly allocate the extra pixels to tiles
# return the 刻度
def get_pixels_ran(num_pixels_x, num_tiles_x):
    tile_pixels_x = np.zeros(num_tiles_x)
    if num_pixels_x % num_tiles_x == 0:
        tile_pixels_x = (num_pixels_x/num_tiles_x)*np.ones(num_tiles_x).astype(int)
    else:
        tile_pixels_x = int(num_pixels_x/num_tiles_x)*np.ones(num_tiles_x).astype(int)
        remain = num_pixels_x % num_tiles_x
        position = np.random.randint(0,num_tiles_x, size = remain)

        for i in range(remain):
            tile_pixels_x[position[i]] += 1

    cum_sum = np.cumsum(tile_pixels_x).astype(int)
    tile_pixels_x = cum_sum - int(tile_pixels_x[0])
    tile_pixels_x = np.append(tile_pixels_x,num_pixels_x)
    return tile_pixels_x
        
        


# This function will calculate the chi_square of pictures
def chi_square(X,Y,bins):
    # First Get the histogram with given bins of
    # Cost too much time 
    hist_x = np.zeros((3,bins))
    hist_y = np.zeros((3,bins))
    d = np.zeros((3,1))

    for i in range(3):
        hist_x[i], cnt = np.histogram(X[:,:,i], bins = bins) #ax[i,0].hist(X[:,:,i])
        hist_x[i] = hist_x[i]/np.sum(hist_x[i])
        hist_y[i], cnt = np.histogram(Y[:,:,i], bins = bins)
        hist_y[i] = hist_y[i]/np.sum(hist_y[i])
        d[i] = np.sum(np.square(hist_x[i] - hist_y[i])/(hist_x[i] + hist_y[i]+1))

    return np.sqrt(np.sum(np.square(d)))

# This chi-square methos compares two histogram
def chi_square_hist(x,y):
    if np.shape(x) == (3,20):
        d = np.zeros((3,1))
        for i in range(3):
            d[i] = np.sum(np.square(x[i] - y[i])/(x[i] + y[i] + 1))
        result = np.sqrt(np.sum(np.square(d)))
    else :
        result = np.sum(np.square(x - y)/(x + y + 1))

    return result




# I is the original image, x and y is the output image
def gauss_subsample(I,x,y):
    I = I.filter(ImageFilter.GaussianBlur)
    I = I.resize((x,y))
    return I 

# get the color label in rgb of each pixel
def color_set(image):
    g_ColorVectorSpace = np.zeros((8,3))

    # BGR模型（不是RGB）
    g_ColorVectorSpace[0] = [0, 0, 0]              #黑色
    g_ColorVectorSpace[1] = [255, 0, 0]            #蓝色
    g_ColorVectorSpace[2] =  [0, 255, 0]             #绿色
    g_ColorVectorSpace[3] =  [255, 255, 0]            #青色
    g_ColorVectorSpace[4] =  [0, 0, 255]              #红色
    g_ColorVectorSpace[5] =  [255, 0, 255]            #洋红色
    g_ColorVectorSpace[6] =  [0, 255, 255]            #黄色
    g_ColorVectorSpace[7] =  [255, 255, 255]          #白色

    g_ColorVectorSpace = g_ColorVectorSpace.astype(int)

    y =np.shape(image)[0]
    x = np.shape(image)[1]

    

    min_d = sys.maxsize
    color = 0
    im_cs = np.zeros((y,x))

    for i in range(y):
        for j in range(x):
            for k in range(8):
                d = np.sqrt(sum(((image[i,j]-g_ColorVectorSpace[k])**2)))
                if d <= min_d:
                    min_d = d
                    color = k

            im_cs[i,j] = k

    return im_cs.astype(int).flatten()


# 通过图像以及分割大小（none or 4 等分）
# 传入的是np.array
def moments(image,divide_num = None):
    def calc_moments(subimage):
        result = np.zeros((9,1))
        for i in range(0,9,3):
            result[i] = np.mean(subimage[:,:,int(i/3)])
            result[1+i] = np.var(subimage[:,:,int(i/3)])
            mu3 = np.mean(subimage[:,:,int(i/3)]**3)
            result[2+i] = (mu3 -3*mu3*result[1+i]**2-mu3**3)/(result[1+i]**3) # 偏度计算公式

        return result


    if divide_num == None:
        return calc_moments(image)
    else:
        y = np.shape(image)[0]
        x = np.shape(image)[1]
        result1 = np.zeros((36,1))
        tmp = np.zeros((9,1))

        
        y_pixels = int(y/2)
        x_pixels = int(x/2)
        cnt = 0
        for i in range(2):
            for j in range(2):
                tmp = calc_moments(image[i*y_pixels:(i+1)*y_pixels , j*x_pixels: (j+1)*x_pixels])
                result1[cnt*9:(cnt+1)*9] += tmp
                cnt += 1

        return result1




In [52]:
def get_hist(image,bins):
    # 假设可以整除
    interval_len = 256/bins
    result_hist = np.zeros(bins**3)
    for i in range(np.shape(image)[0]):
        for j in range(np.shape(image)[1]):
            result = 0
            for k in range(3):
                result = result + int(image[i,j,k]/interval_len) * (bins**k)
                
            result_hist[result] += 1

    return result_hist.astype(int)

    

In [136]:
def hist_rgb(target, result, dict_pics, tiles_x, tiles_y,x1,x2,y1,y2):
    bins = 20
    top_k_num = 10
    
    # ============================================get hist==========================================
    for key,pic in dict_pics.items():
        pic_hist = np.zeros((3,bins))
        for i in range(3):
            tmp = np.histogram(pic[:,:,i], range = (0,255) ,density =True, bins = bins)[0]
            pic_hist[i] = tmp
        
        dict_pics[key] = pic_hist

    # # =======================================get target pic=================================================
    print("All histogram have been generated. Begin to generate the picture~~~~")
    i = 0
    for i in tqdm(range(tiles_y)):
        for j in range(tiles_x):
            # Get target part of pic
            target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1]]
            # if (i == 0) & (j == 0):
            #     target_ij = target[y1[i]:y1[i+2],x1[j]:x1[j+2]]

            # elif (i == 0) & (j != 0) & (j != tiles_x - 1):
            #     target_ij = target[y1[i]:y1[i+2],x1[j-1]:x1[j+2]]

            # elif (i == 0) & (j == tiles_x - 1):
            #     target_ij = target[y1[i]:y1[i+2],x1[j-1]:x1[j+1]]

            # elif (i != 0) & (i != tiles_y -1) & (j == 0):
            #     target_ij = target[y1[i-1]:y1[i+2],x1[j]:x1[j+2]]

            # elif (i == tiles_y - 1) & (j == 0):
            #     target_ij = target[y1[i-1]:y1[i+1],x1[j]:x1[j+2]]

            # elif (i == tiles_y - 1) & (j != 0) & (j != tiles_x - 1):
            #     target_ij = target[y1[i-1]:y1[i+1],x1[j-1]:x1[j+2]]
            
            # elif (i == tiles_y - 1) & (j == tiles_x -1):
            #     target_ij = target[y1[i-1]:y1[i+1],x1[j-1]:x1[j+1]]

            # elif (i != 0) & (i != tiles_y -1) & (j == tiles_x - 1):
            #     target_ij = target[y1[i-1]:y1[i+2],x1[j-1]:x1[j+1]]
        
            # else:
            #     target_ij = target[y1[i-1]:y1[i+2],x1[j-1]:x1[j+2]]

            y_pixels = y2[i+1] - y2[i]
            x_pixels = x2[j+1] - x2[j]

            # compare target_ij with the image
            # First just read one then compare one and get the top 10
            dict_path = {}
            heap = []


            # ==================================Get chi distance======================================

            # ============Method1: Get the RGB distance then using average to get the best==============
            # get histogram of target pictures
            target_ij_hist = np.zeros((3,bins))
            for k in range(3):
                tmp = np.histogram(target_ij[:,:,k],range = (0,255), bins = bins, density=True)[0]
                target_ij_hist[k] = tmp

            for item in dict_pics.items():
                # input vector shape (3,bins)
                distance = chi_square_hist(target_ij_hist,item[1])
                dict_path[item[0]] = distance

            # # ===============Method2: Using one histogram（TOO SLOW!）=================
            # # Create the hist
            # target_ij_hist2 = get_hist(target,8)

            # # Get the hist of pics
            # for item in dict_pics.items():
            #     dict_path[item[0]] = chi_square_hist(target_ij_hist2,item[1])
            

            # ========================================Get top k===============================================
            topk = heapq.nsmallest(top_k_num, dict_path.items(), key = lambda x : x[1])
            # print("Get the patch of %s, %s" % (i,j))
            # print(top)
            # =================================From topk to get the top 1=====================================
            
            dim = [x_pixels, y_pixels]
            target_ij = cv.resize(target_ij,dim)
        
            # ----首先得到颜色集比较----
            # tgt_cs = color_set(target_ij)
         
            # d_max = 0
            # top = 0
            
            # for k in range(top_k_num):
            #     tmp = cv.imread(topk[k][0])
            #     tmp = cv.resize(tmp,dim)
            #     tmp_cs = color_set(tmp)
            #     # using cosin
            #     d = np.abs(np.dot(tgt_cs,tmp_cs))/(np.linalg.norm(tgt_cs) * np.linalg.norm(tmp_cs))
            #     print(d)
            #     if d > d_max:
            #         d_max = d
            #         top = k


            # 

            # ----直接比较整个图的像素----
            
            # max_result = 0
            # top = 0
            # k = 0
            # for k in range(top_k_num):
            #     sum_results = 0
            #     tmp = cv.imread(topk[k][0])
            #     tmp = cv.resize(tmp,dim)
            #     for m in range(y_pixels):
            #         for n in range(x_pixels):
            #             sum_results = sum_results + np.abs(np.dot(target_ij[m,n],tmp[m,n]))/(np.linalg.norm(target_ij[m,n]) * np.linalg.norm(tmp[m,n]))
            # TODO: 这个要优化一下
            #     if sum_results>max_result:
            #         max_result = sum_results
            #         top = k

            # ---通过比较低阶矩-----
            max_result = 0
            top = 0
            k = 0

            target_moments = moments(target_ij)

            for k in range(top_k_num):
                tmp = cv.imread(topk[k][0])
                tmp = cv.resize(tmp,dim)
                tmp_moments = moments(tmp)
                d = np.abs(np.dot(target_moments.T, tmp_moments))/(np.linalg.norm(target_moments)*np.linalg.norm(tmp_moments))

                if d>max_result:
                    max_result = d
                    top = k





            # ==============================Process the best image into proper size===========================
            # Then fit the image into the proper size: Using Gaussian subsample
            # First get the Gaussian smoothing
            # Then get the subsample with the proper size
            
            im = Image.open(topk[top][0])
            im = gauss_subsample(im,x = x_pixels, y = y_pixels)

            # 向结果矩阵中拼接
            result[y2[i]: y2[i+1], x2[j]: x2[j+1]] = np.array(im)

    return result


In [134]:
# I need to compare 3 different distance to get a better one
# Using opencv to implemnent
# All this histogram is hsv
def hist_compare(target, result, dict_pics, tiles_x, tiles_y,x1,x2,y1,y2,compare_method):
    dict_hists = {}
    channels = [0, 1]
    h_bins = 15
    s_bins = 20
    histSize = [h_bins, s_bins]
    # hue varies from 0 to 179, saturation from 0 to 255
    h_ranges = [0, 180]
    s_ranges = [0, 256]
    ranges = h_ranges + s_ranges # concat lists

    # 计算出datasets中的histogram
    for key, pic in dict_pics.items():
        hsv = cv.cvtColor(pic, cv.COLOR_BGR2HSV)
        hist = cv.calcHist([hsv], channels, None, histSize, ranges, accumulate=False)
        cv.normalize(hist, hist, alpha=0, beta=1, norm_type=cv.NORM_MINMAX)
        dict_hists[key] = hist

    print("All histogram have been generated. Begin to generate the picture~~~~")
    

    # 
    for i in tqdm(range(tiles_y)):
        for j in range(tiles_x):    
            # Get target part of pic
            target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1]]
            y_pixels = y2[i+1] - y2[i]
            x_pixels = x2[j+1] - x2[j]

            # compare target_ij with the image
            # First just read one then compare one and get the top 10
            dict_path = {}
            heap = []


            # Get the histogram of the target image
            hsv = cv.cvtColor(target_ij, cv.COLOR_BGR2HSV)
            hist_target = cv.calcHist([hsv], channels, None, histSize, ranges, accumulate=False)
            cv.normalize(hist_target, hist_target, alpha=0, beta=1, norm_type=cv.NORM_MINMAX)


            # =====using different methods====
            # https://docs.opencv.org/4.x/d6/dc7/group__imgproc__hist.html#ga994f53817d621e2e4228fc646342d386
            for key, hist in dict_hists.items():
                dict_path[key] = cv.compareHist(hist_target, hist , compare_method)

            # get topk
            if (compare_method == cv.HISTCMP_CORREL) or (compare_method == cv.HISTCMP_INTERSECT):
                top = heapq.nlargest(1,dict_path.items(), key = lambda x : x[1])
            else:
                top = heapq.nsmallest(1,dict_path.items(), key = lambda x : x[1])

            im = Image.open(top[0][0])
            im = gauss_subsample(im,x = x_pixels, y = y_pixels)

            result[y2[i]: y2[i+1], x2[j]: x2[j+1]] = np.array(im)

        

    return result





In [43]:
# I need to compare 3 different distance to get a better one
# Using opencv to implemnent
# All this histogram is rgb
def hist_comp_rgb(target, result, dict_pics, tiles_x, tiles_y,x1,x2,y1,y2,compare_method):
    dict_hists = {}
    channels = [0, 1, 2]
    r_bins = 20
    g_bins = 20
    b_bins = 20

    histSize = [r_bins, g_bins, b_bins]
    # hue varies from 0 to 179, saturation from 0 to 255
    r_ranges = [0, 256]
    g_ranges = [0, 256]
    b_ranges = [0, 256]
    ranges = r_ranges + g_ranges + b_ranges# concat lists

    # 计算出datasets中的histogram
    for key, pic in dict_pics.items():
        hist = cv.calcHist([pic], channels, None, histSize, ranges, accumulate=False)
        cv.normalize(hist, hist, alpha=0, beta=1, norm_type=cv.NORM_MINMAX)
        dict_hists[key] = hist

    print("All histogram have been generated. Begin to generate the picture~~~~")
    

    # 扩大histogram看分布会不会更加准确
    
    for i in tqdm(range(tiles_y)):
        for j in range(tiles_x):    
            # Get target part of pic
            
            

            #target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1]]

            if (i == 0) & (j == 0):
                target_ij = target[y1[i]:y1[i+2],x1[j]:x1[j+2]]

            elif (i == 0) & (j != 0) & (j != tiles_x - 1):
                target_ij = target[y1[i]:y1[i+2],x1[j-1]:x1[j+2]]

            elif (i == 0) & (j == tiles_x - 1):
                target_ij = target[y1[i]:y1[i+2],x1[j-1]:x1[j+1]]

            elif (i != 0) & (i != tiles_y -1) & (j == 0):
                target_ij = target[y1[i-1]:y1[i+2],x1[j]:x1[j+2]]

            elif (i == tiles_y - 1) & (j == 0):
                target_ij = target[y1[i-1]:y1[i+1],x1[j]:x1[j+2]]

            elif (i == tiles_y - 1) & (j != 0) & (j != tiles_x - 1):
                target_ij = target[y1[i-1]:y1[i+1],x1[j-1]:x1[j+2]]
            
            elif (i == tiles_y - 1) & (j == tiles_x -1):
                target_ij = target[y1[i-1]:y1[i+1],x1[j-1]:x1[j+1]]

            elif (i != 0) & (i != tiles_y -1) & (j == tiles_x - 1):
                target_ij = target[y1[i-1]:y1[i+2],x1[j-1]:x1[j+1]]
        
            else:
                target_ij = target[y1[i-1]:y1[i+2],x1[j-1]:x1[j+2]]



            # compare target_ij with the image
            # First just read one then compare one and get the top 10
            dict_path = {}
            heap = []


            # Get the histogram of the target image
            hist_target = cv.calcHist([target_ij], channels, None, histSize, ranges, accumulate=False)
            cv.normalize(hist_target, hist_target, alpha=0, beta=1, norm_type=cv.NORM_MINMAX)


            # =====using different methods====
            # https://docs.opencv.org/4.x/d6/dc7/group__imgproc__hist.html#ga994f53817d621e2e4228fc646342d386
            for key, hist in dict_hists.items():
                dict_path[key] = cv.compareHist(hist_target, hist , compare_method)

            # get topk
            if (compare_method == cv.HISTCMP_CORREL) or (compare_method == cv.HISTCMP_INTERSECT):
                top = heapq.nlargest(1,dict_path.items(), key = lambda x : x[1])
            else:
                top = heapq.nsmallest(1,dict_path.items(), key = lambda x : x[1])

            im = Image.open(top[0][0])

            y_pixels = y2[i+1] - y2[i]
            x_pixels = x2[j+1] - x2[j]

            im = gauss_subsample(im,x = x_pixels, y = y_pixels)

            result[y2[i]: y2[i+1], x2[j]: x2[j+1]] = np.array(im)


        

    return result


            




In [95]:
def cosin(target, result,dict_pics, tiles_x, tiles_y,x1,x2,y1,y2,size):
    print("I entered the cosin method!!!!~")
    # ===========================将图片标准化========================
    def get_thum(image, size=(size, size), greyscale=False):
    # 利用image对图像大小重新设置, Image.ANTIALIAS为高质量的
        image = Image.fromarray(image)
        image = image.resize(size, Image.ANTIALIAS)
        if greyscale:
            # 将图片转换为P模式，其每个像素用8个bit表示
            image = image.convert('P')
        return image


    def get_cosin(image1,pic_array):
        image1_array = np.array(get_thum(image1)).flatten()
        return np.dot(image1_array,pic_array.T)/(np.linalg.norm(image1_array)*np.linalg.norm(pic_array))
        


    # 首先将1000张图片进行一个压缩，变成灰度图像
    for key, pic in dict_pics.items():
        dict_pics[key] = np.array(get_thum(pic)).flatten()

    print("All pictures have been converted to 'P'. Start to generate picture!!!!~")
    for i in range(tiles_y):
        for j in range(tiles_x):
            # 进度条
            if (i == int(tiles_y/2)) & (j == int(tiles_x/2)):
                print("Have already Finished 1/4!! ~~~~")

            # Get target part of pic
            target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1]]
            y_pixels = y2[i+1] - y2[i]
            x_pixels = x2[j+1] - x2[j]

            # compare target_ij with the image
            # First just read one then compare one and get the top 10
            dict_path = {}
            heap = []
            # =============================获得余弦值==============================
            for key, pic_array in dict_pics.items():
                dict_path[key] = get_cosin(target_ij,pic_array)
            
            # ============================得到top===========================
            top = heapq.nlargest(1,dict_path.items(), key = lambda x:x[1])
            im = Image.open(top[0][0])
            im = gauss_subsample(im,x = x_pixels, y = y_pixels)

            # 向结果矩阵中拼接
            result[y2[i]: y2[i+1], x2[j]: x2[j+1]] = np.array(im)

    return result




In [87]:
# First give the picture different color category 

### Using hash to get the similar picture

In [133]:
def ahash_compare(target, result,dict_pics, tiles_x, tiles_y,x1,x2,y1,y2):
    print("I haved entered ahash_compare methos!!!~~")
    
    def ahash(image):
        # 将图片缩放为8*8的
        image = Image.fromarray(image)
        # image = cv.resize(image, (8, 8), interpolation=cv.INTER_CUBIC)
        image = np.array(image.resize((8, 8)))
        # 将图片转化为灰度图
        gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
        # gray = np.array(image.convert('L'))
        # s为像素和初始灰度值，hash_str为哈希值初始值
        s = 0
        # 遍历像素累加和
        for i in range(8):
            for j in range(8):
                s = s + gray[i, j]
        # 计算像素平均值
        avg = s / 64
        # 灰度大于平均值为1相反为0，得到图片的平均哈希值，此时得到的hash值为64位的01字符串
        ahash_str = ''
        for i in range(8):
            for j in range(8):
                if gray[i, j] > avg:
                    ahash_str = ahash_str + '1'
                else:
                    ahash_str = ahash_str + '0'
        result = ''
        for i in range(0, 64, 4):
            result += ''.join('%x' % int(ahash_str[i: i + 4], 2))
        # print("ahash值：",result)
        return result

    # phash
    def phash(image):
        # 加载并调整图片为32*32的灰度图片
        # image = Image.fromarray(image)
        image = cv.cvtColor(image,cv.COLOR_RGB2GRAY)
        image = cv.resize(image,(32,32))
        # image = Image.fromarray(image)
        # image = image.resize((32,32))
        # image = image.convert('L')
        
        # 创建二维列表
        h, w = image.shape[:2]
        vis0 = np.zeros((h, w), np.float32)
        vis0[:h, :w] = image

        # DCT二维变换
        # 离散余弦变换，得到dct系数矩阵
        img_dct = cv.dct(cv.dct(vis0))
        img_dct = img_dct.resize(8,8)
        # 把list变成一维list
        img_list = np.array(img_dct).flatten()
        
        # 计算均值
        img_mean = np.mean(img_list)
        avg_list = ['0' if i<img_mean else '1' for i in img_list]
        return ''.join(['%x' % int(''.join(avg_list[x:x+4]),2) for x in range(0,64,4)])

    # 差异值哈希算法
    def dhash(image):
        # 将图片转化为8*8
        # image = Image.fromarray(image)
        # image = image.resize((9,8))
        # gray = np.array(image.convert('L'))

        image = cv.resize(image, (9, 8), interpolation=cv.INTER_CUBIC)
        # 将图片转化为灰度图
        gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
        dhash_str = ''
        for m in range(8):
            for n in range(8):
                if gray[m , n] > gray[m, n + 1]:
                    dhash_str = dhash_str + '1'
                else:
                    dhash_str = dhash_str + '0'
        result = ''
        for m in range(0, 64, 4):
            result += ''.join('%x' % int(dhash_str[m: m + 4], 2))
        # print("dhash值",result)
        return result


    # 计算两个哈希值之间的差异
    def compHash(hash1, hash2):
        n = 0
        # hash长度不同返回-1,此时不能比较
        if len(hash1) != len(hash2):
            return -1
        # 如果hash长度相同遍历长度
        for i in range(len(hash1)):
            if hash1[i] != hash2[i]:
                n = n + 1
        return n

    # =====首先得到1000张图片的ahash====
    # for key, pic in dict_pics.items():
    #     dict_pics[key] = ahash(pic)
    for key, pic in dict_pics.items():
        dict_pics[key] = dhash(pic)
    
    # =========开始检索========
    print("All hash code have been obtained! Begin to generate!!!~")
    for i in range(tiles_y):
        for j in range(tiles_x):
            # 进度条
            if (i == int(tiles_y/2)) & (j == int(tiles_x/2)):
                print("Have already Finished 1/4!! ~~~~")

            # Get target part of pic
            target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1]]
            y_pixels = y2[i+1] - y2[i]
            x_pixels = x2[j+1] - x2[j]

            # compare target_ij with the image
            # First just read one then compare one and get the top 10
            dict_path = {}
            heap = []
             
            for key, hash_value in dict_pics.items():
                dict_path[key] = compHash(dhash(target_ij),hash_value)

            top = heapq.nsmallest(1,dict_path.items(), key = lambda x:x[1])

            im = Image.open(top[0][0])
            im = gauss_subsample(im,x = x_pixels, y = y_pixels)

            # 向结果矩阵中拼接
            result[y2[i]: y2[i+1], x2[j]: x2[j+1]] = np.array(im)

    return result
 

In [127]:
def lab_compare(target, result,dict_pics, tiles_x, tiles_y,x1,x2,y1,y2):
    # calculate lab color distance
    def calculate_color_distance(color1, color2):
        rmean = (int(color1[0]) - int(color2[0]))/2
        r = int(color1[0]) - int(color2[0])
        g = int(color1[1]) - int(color2[1])
        b = int(color1[2]) - int(color2[2])
        distance = np.sqrt(((2+rmean/256)*r*r + 4*g*g + (2+(255-rmean)/256)*b*b))
        return distance

    # First resize the pics into corresponding size of 
    x = int(np.shape(target)[1]/tiles_x)
    y = int(np.shape(target)[0]/tiles_y)

    # Then resize pics dataset into this size
    for key, pics in dict_pics.items():
        tmp = Image.fromarray(pics)
        tmp = np.array(tmp.resize((x,y)))
        dict_pics[key] = tmp

    # begin to compare each pixels, 直接忽略边缘
    print("All pictures have been resized into proper size! Begin to generate!!!~")
    for i in range(tiles_y):
        for j in range(tiles_x):
            # 进度条
            if (i == int(tiles_y/2)) & (j == int(tiles_x/2)):
                print("Have already Finished 1/4!! ~~~~")

            # Get target part of pic
            target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1]]
            y_pixels = y2[i+1] - y2[i]
            x_pixels = x2[j+1] - x2[j]

            # compare target_ij with the image
            # First just read one then compare one and get the top 10
            dict_path = {}
            heap = []

            # compute the distance of each pixels
            # 这个地方或许可以优化？
            
            for key, pics in dict_pics.items():
                sum_lab = 0
                for m in range(y):
                    for n in range(x):
                        sum_lab += calculate_color_distance(target_ij[m,n],pics[m,n])
                
                dict_path[key] = sum_lab

            # get the smallest
            top = heapq.nsmallest(10,dict_path.items(), key = lambda x:x[1])

            im = Image.open(top[0][0])
            im = gauss_subsample(im,x = x_pixels, y = y_pixels)

            # 向结果矩阵中拼接
            result[y2[i]: y2[i+1], x2[j]: x2[j+1]] = np.array(im)
        
    return result
                
                    
    
    

### Start to generate the pictures

In [44]:
def main(target,pixels_x, pixels_y, tiles_x,tiles_y,method = None):
    # first get the partition of original pic
    # x1,y1 表示原始图像中的分割
    # x2,y2 表示输出图像中的分割大小，找到对应图片后应该压缩到的大小
    bins = 20
    x1 = get_pixels_ran(np.shape(target[:,:,0])[1], tiles_x).astype(int)
    y1 = get_pixels_ran(np.shape(target[:,:,0])[0], tiles_y).astype(int)

    # construct x and y with the size of tiles
    x2 = get_pixels_ran(pixels_x, tiles_x).astype(int)
    y2 = get_pixels_ran(pixels_y, tiles_y) .astype(int)

    # result image
    result = np.zeros((pixels_y, pixels_x, 3))

    # ===========================获得每张图片==========================
    # -------------纯色图片测试集------------
#     list_color = ['#47493c','#cdcaa8','#494630','#7c95ac','#7c95ac','#7c95ac','#506b8b','#19180e','#748570','#241b14','#b48c2c','#6495ED',
# '#87CEFA','#90EE90','#F5F5DC','#FFE4B5','#FFFFFF','#800000']
#     dictMerged = {}
#     for color in list_color:
#         path = 'data/pure_color/' + color + '.jpg'
#         im = Image.open(path)
#         im = np.array(im)
#         dictMerged[path] = im
        
    # #########1000张training图片集########
    # 将所有图片读成两个数组：man/natural

    with open('data/dataset/manmade_training.txt','r',encoding='utf-8') as f:
        im_name_m = [line.strip('\n') for line in f.readlines()]
    with open('data/dataset/natural_training.txt','r',encoding='utf-8') as f:
        im_name_n = [line.strip('\n') for line in f.readlines()]

    

    # 将图片存储字典中, 字典中存储的就是image对象
    dict_images_m = {}
    dict_images_n = {}
    cnt = 0
    for path in im_name_m:
        cnt += 1
        path = 'data/dataset/' + path
        im = cv.imread(path)
        #im = np.array(im)
        dict_images_m[path] = im
        if cnt > 20:
            break

    cnt = 0
    for path in im_name_n:
        cnt += 1
        path = 'data/dataset/' + path
        im = cv.imread(path)
        #im = np.array(im)
        dict_images_n[path] = im
        if cnt> 20:
            break

    dictMerged = dict( dict_images_m, **dict_images_n )
    
    # ==============================Using different method to find the result===========================
    result = hist_rgb(target,result,dictMerged,tiles_x,tiles_y,x1,x2,y1,y2)
    # cv.HISTCMP_CORREL  cv.HISTCMP_CHISQR   cv.HISTCMP_INTERSECT    cv.HISTCMP_BHATTACHARYYA   cv.HISTCMP_KL_DIV
    # result = hist_compare(target,result,dictMerged,tiles_x,tiles_y,x1,x2,y1,y2, method)
    # result = hist_comp_rgb(target,result,dictMerged,tiles_x,tiles_y,x1,x2,y1,y2, method)
     
    # # cosin method
    # size = min(int(np.shape(target[:,:,0])[1]/tiles_x),int(np.shape(target[:,:,0])[0]/tiles_y))
    # result = cosin(target,result,dictMerged,tiles_x,tiles_y,x1,x2,y1,y2,size)
          
    # result = ahash_compare(target,result,dictMerged,tiles_x,tiles_y,x1,x2,y1,y2) 
    # result = lab_compare(target,result,dictMerged,tiles_x,tiles_y,x1,x2,y1,y2) 
    return result



            

In [45]:
def store_result(path,num,pixels_x,pixels_y, tile_x,tile_y,pic_name,method = None):
    dir = 'results/results%s/' % num + pic_name
    #result = main(np.array(Image.open(path)), pixels_x,pixels_y,tile_x,tile_y)
    result = main(cv.imread(path), pixels_x,pixels_y,tile_x,tile_y,method = method)
    
    Image.fromarray(np.uint8(result)).save(dir)

In [139]:
store_result('data/target/target1.jpg',1, 1750,1250, 275,200, 'moments1.jpg')

All histogram have been generated. Begin to generate the picture~~~~


  0%|          | 0/200 [00:00<?, ?it/s]/var/folders/ph/35nx213151gfc98gbz9pxw580000gn/T/ipykernel_1160/4060470536.py:132: RuntimeWarning: divide by zero encountered in divide
  result[2+i] = (mu3 -3*mu3*result[1+i]**2-mu3**3)/(result[1+i]**3) # 偏度计算公式
/var/folders/ph/35nx213151gfc98gbz9pxw580000gn/T/ipykernel_1160/1148401957.py:137: RuntimeWarning: invalid value encountered in divide
  d = np.abs(np.dot(target_moments.T, tmp_moments))/(np.linalg.norm(target_moments)*np.linalg.norm(tmp_moments))
/var/folders/ph/35nx213151gfc98gbz9pxw580000gn/T/ipykernel_1160/4060470536.py:132: RuntimeWarning: invalid value encountered in divide
  result[2+i] = (mu3 -3*mu3*result[1+i]**2-mu3**3)/(result[1+i]**3) # 偏度计算公式
 54%|█████▍    | 108/200 [1:06:29<1:00:48, 39.66s/it]

In [91]:
list_method = [cv.HISTCMP_CORREL,  cv.HISTCMP_CHISQR,   cv.HISTCMP_INTERSECT,    cv.HISTCMP_BHATTACHARYYA,  cv.HISTCMP_KL_DIV,cv.HISTCMP_CHISQR_ALT]
for i in range(len(list_method)):
    store_result('data/target/target1.jpg',1, 1900,1250,150,80,'comp_hist_rgb%s.jpg' % (i+1),list_method[i])

All histogram have been generated. Begin to generate the picture~~~~


  0%|          | 0/80 [00:00<?, ?it/s]


IndexError: index 12 is out of bounds for axis 0 with size 12

### Script

This part is used to test some methods. 还有生成纯色测试集的代码

In [ ]:
def hex2rgb(hex):
    r = int('0x' + hex[1:3],16)
    g = int('0x' + hex[3:5],16)
    b = int('0x' + hex[5:],16)
    return (r,g,b)

list_color = ['#47493c','#cdcaa8','#494630','#7c95ac','#7c95ac','#7c95ac','#506b8b','#19180e','#748570','#241b14','#b48c2c','#6495ED',
'#87CEFA','#90EE90','#F5F5DC','#FFE4B5','#FFFFFF','#800000']
for i in range(len(list_color)):
    Image.new("RGB",size = (256,256),color = hex2rgb(list_color[i])).save('data/pure_color/%s.jpg' % (list_color[i]))
    

In [1]:
h_ranges = [0, 180]
s_ranges = [0, 256]
ranges = h_ranges + s_ranges
ranges

[0, 180, 0, 256]

In [24]:
# test wehether cv image objects can be converted into array
image = cv.imread('data/target/target2.jpg')
# numpy.ndarray!!

In [128]:
image = cv.resize(image,[11,11], cv.INTER_AREA)
#cv.imshow(image)
color_set(image)


array([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7])

In [58]:
get_hist(image,4)

array([20,  8,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0, 10,  9,  0,  0,  1, 10,  9,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  6,  4,  2,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 16])

In [76]:
test = {'a':10,'b':10,'c':0, 'd':4}
heapq.nsmallest(3,test.items(), lambda x:x[1])

[('c', 0), ('d', 4), ('a', 10)]

In [109]:
for i in range(0,9,3):
    print(i)

0
3
6


In [131]:
moments(image,1)
#np.shape(image)


array([[ 8.04000000e+01],
       [ 2.67976000e+03],
       [-1.32545694e-01],
       [ 6.84000000e+01],
       [ 2.49528000e+03],
       [-1.25879260e-01],
       [ 6.01600000e+01],
       [ 2.28045440e+03],
       [-1.89370716e-01],
       [ 1.88400000e+02],
       [ 7.40960000e+02],
       [-4.58907098e-01],
       [ 1.56520000e+02],
       [ 9.04009600e+02],
       [-3.78836495e-01],
       [ 1.28200000e+02],
       [ 1.79032000e+03],
       [-1.86507718e-01],
       [ 8.20000000e+01],
       [ 1.71952000e+03],
       [-1.78725612e-01],
       [ 7.16400000e+01],
       [ 2.01351040e+03],
       [-1.80379536e-01],
       [ 6.88800000e+01],
       [ 1.98378560e+03],
       [-1.97118225e-01],
       [ 1.66040000e+02],
       [ 1.34027840e+03],
       [-2.61107676e-01],
       [ 1.70080000e+02],
       [ 9.02313600e+02],
       [-3.37879059e-01],
       [ 1.49360000e+02],
       [ 1.41791040e+03],
       [-2.30309142e-01]])